In [ ]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import json
import gc
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import re
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from tqdm import tqdm

In [ ]:
df = pd.read_csv("/content/bbc-text.csv")
df.head(10)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...
7,sport,henman hopes ended in dubai third seed tim hen...
8,sport,wilkinson fit to face edinburgh england captai...
9,entertainment,last star wars not for children the sixth an...


In [ ]:
import nltk
from nltk.corpus import stopwords
import string

# Load the stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def clean_text(text, stop_words):
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text
# Clean the text data
df['text'] = df['text'].apply(clean_text, stop_words=stop_words)

# Save the cleaned dataset
df.to_csv('bbc-text-cleaned.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv("/content/bbc-text-cleaned.csv")
df.head()

,category,text
0,tech,tv future hands viewers home theatre systems p...
1,business,worldcom boss left books alone former worldcom...
2,sport,tigers wary farrell gamble leicester say rushe...
3,sport,yeading face newcastle fa cup premiership side...
4,entertainment,ocean twelve raids box office ocean twelve cri...


In [ ]:
df.shape

(2225, 2)

In [ ]:
category_count = df['category'].value_counts()

categories = category_count.index

categories

Index(['sport', 'business', 'politics', 'tech', 'entertainment'], dtype='object')

In [ ]:
df['encoded_text'] = df['category'].astype('category').cat.codes

df.head(10)

,category,text,encoded_text
0,tech,tv future hands viewers home theatre systems p...,4
1,business,worldcom boss left books alone former worldcom...,0
2,sport,tigers wary farrell gamble leicester say rushe...,3
3,sport,yeading face newcastle fa cup premiership side...,3
4,entertainment,ocean twelve raids box office ocean twelve cri...,1
5,politics,howard hits back mongrel jibe michael howard s...,2
6,politics,blair prepares name poll date tony blair likel...,2
7,sport,henman hopes ended dubai third seed tim henman...,3
8,sport,wilkinson fit face edinburgh england captain j...,3
9,entertainment,last star wars children sixth final star wars ...,1


In [ ]:
data_texts = df['text'].to_list()

data_labels = df['encoded_text'].to_list()

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.2, random_state = 42)


train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 42)


In [ ]:
 tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

val_encodings = tokenizer(val_texts, truncation = True, padding = True )

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file tf_model.h5 from cache at /root/.cache/huggingface/hub/models--

In [ ]:
max_len = 148

model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0, input_shape=(max_len,)),
    tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=32, input_length=max_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

train_sequences = [tokenizer.encode(text, max_length=max_len, truncation=True) for text in train_texts]
train_sequences_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len, padding='post')

train_labels_onehot = tf.keras.utils.to_categorical(train_labels, num_classes=5)

model.fit(train_sequences_padded, train_labels_onehot, epochs=50, batch_size=32)


Epoch 1/50
56/56 [==============================] - 1s 14ms/step - loss: 1.6112 - accuracy: 0.1884
Epoch 2/50
56/56 [==============================] - 1s 14ms/step - loss: 1.5985 - accuracy: 0.2798
Epoch 3/50
56/56 [==============================] - 1s 14ms/step - loss: 1.5866 - accuracy: 0.3854
Epoch 4/50
56/56 [==============================] - 1s 15ms/step - loss: 1.5746 - accuracy: 0.4841
Epoch 5/50
56/56 [==============================] - 1s 15ms/step - loss: 1.5622 - accuracy: 0.5732
Epoch 6/50
56/56 [==============================] - 1s 14ms/step - loss: 1.5492 - accuracy: 0.6413
Epoch 7/50
56/56 [==============================] - 1s 14ms/step - loss: 1.5359 - accuracy: 0.7072
Epoch 8/50
56/56 [==============================] - 1s 14ms/step - loss: 1.5221 - accuracy: 0.7622
Epoch 9/50
56/56 [==============================] - 1s 14ms/step - loss: 1.5077 - accuracy: 0.7985
Epoch 10/50
56/56 [==============================] - 1s 16ms/step - loss: 1.4927 - accuracy: 0.8331
Epoch 11/

In [ ]:
test_text = test_texts[7]
test_text

'software watching work software monitor every keystroke action performed pc also used legally binding evidence wrongdoing unveiled worries cybercrime sabotage prompted many employers consider monitoring employees developers behind system claim breakthrough way data monitored stored privacy advocates concerned invasive nature software system joint venture security firm 3ami storage specialists bridgehead software joined forces create system monitor computer activity store retrieve disputed files within minutes firms finding deep water result data misuse sabotage data theft commonly committed within organisation according national hitech crime unit nhtcu survey conducted behalf nop found evidence 80 medium large companies victims form cybercrime bridgehead software come techniques prove legal standard stored file pc tampered ironically impetus developing system came result freedom information act requires companies store data certain amount time storage system incorporated application d

In [ ]:
predict_input = [tokenizer.encode(text, max_length=max_len, truncation=True) for text in [test_text]]
predict_input_padded = tf.keras.preprocessing.sequence.pad_sequences(predict_input, maxlen=max_len, padding='post')
output = model(predict_input_padded)[0]
prediction_value = tf.argmax(output, axis=-1).numpy()
prediction_value 


4

In [ ]:

# Prepare test data
test_sequences = [tokenizer.encode(text, max_length=max_len, truncation=True) for text in test_texts]
test_sequences_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_len, padding='post')
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, num_classes=5)

# Make predictions on test data
y_pred = model.predict(test_sequences_padded)
y_pred = np.argmax(y_pred, axis=1)

# Calculate metrics
accuracy = np.mean(y_pred == test_labels)
report = classification_report(test_labels, y_pred, digits=3)

print(f"Accuracy: {accuracy:.3f}")
print(f"Classification report:\n{report}")

1/1 [==============================] - 0s 67ms/step
Accuracy: 0.889
Classification report:
              precision    recall  f1-score   support

           0      1.000     0.750     0.857         4
           1      1.000     0.667     0.800         3
           2      0.750     1.000     0.857         3
           3      1.000     1.000     1.000         4
           4      0.800     1.000     0.889         4

    accuracy                          0.889        18
   macro avg      0.910     0.883     0.881        18
weighted avg      0.914     0.889     0.886        18

